# 🛫 FlightAI - Airline Assistant
This Jupyter Notebook contains a chatbot implementation that supports OpenAI's GPT-4 and Llama 3 via Ollama.
It can retrieve ticket prices and engage in conversations dynamically.

In [ ]:

import os
import re
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import ollama

# Load environment variables
load_dotenv()

# Set OpenAI API key (if using OpenAI)
openai_api_key = os.getenv('OPENAI_API_KEY')

# Model selection: OpenAI GPT or Local Llama
USE_OPENAI = bool(openai_api_key)  # Auto-detect OpenAI API key
OPENAI_MODEL = "gpt-4o-mini"  # Use GPT model with function calling
OLLAMA_MODEL = "llama3"  # Change this to match the local Ollama model

# Initialize OpenAI client (only if OpenAI API is available)
if USE_OPENAI:
    openai = OpenAI()

# System message for chatbot
SYSTEM_MESSAGE = (
    "You are a helpful assistant for an Airline called FlightAI. "
    "Give short, courteous answers, no more than 1 sentence. "
    "Always be accurate. If you don't know the answer, say so."
)

# Ticket Prices
TICKET_PRICES = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}


In [ ]:

# Function to fetch ticket prices
def get_ticket_price(destination_city):
    city = destination_city.lower()
    return TICKET_PRICES.get(city, "Unknown")

# Detect if user query is related to ticket pricing
def detect_ticket_query(message):
    price_patterns = [
        r"how much is a ticket to (\w+)\??",
        r"ticket price for (\w+)\??",
        r"cost of a flight to (\w+)\??",
        r"price to (\w+)\??"
    ]
    for pattern in price_patterns:
        match = re.search(pattern, message, re.IGNORECASE)
        if match:
            return match.group(1)
    return None


In [ ]:

# OpenAI function definition
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {"type": "string", "description": "The city the customer wants to travel to"},
        },
        "required": ["destination_city"]
    }
}


In [ ]:

def chat_openai(message, history):
    messages = [{"role": "system", "content": SYSTEM_MESSAGE}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages, tools=[{"type": "function", "function": price_function}])
    
    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        tool_call = message.tool_calls[0]
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        tool_response = {"role": "tool", "content": json.dumps({"destination_city": city, "price": price}), "tool_call_id": tool_call.id}
        messages.append(tool_response)
        response = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages)
    
    return response.choices[0].message.content


In [ ]:

def chat_ollama(message, history):
    city = detect_ticket_query(message)
    if city:
        price = get_ticket_price(city)
        return f"A return ticket to {city.capitalize()} costs {price}."
    history_text = "\n".join([f"{h['role']}: {h['content']}" for h in history])
    formatted_prompt = f"{SYSTEM_MESSAGE}\n\n{history_text}\n\nUser: {message}\nAssistant:"
    response = ollama.chat(model=OLLAMA_MODEL, messages=[{"role": "user", "content": formatted_prompt}])
    return response["message"]["content"]


In [ ]:

def chat(message, history):
    return chat_openai(message, history) if USE_OPENAI else chat_ollama(message, history)


In [ ]:

# Launch Gradio UI for chatbot interaction
gr.ChatInterface(fn=chat, type="messages").launch()
